In [1]:
import bz2
import xml.sax
import mwparserfromhell as mph

In [2]:
base_path = '/Users/harangju/Developer/data/wiki/'
xml_name = 'enwiki-20190801-pages-articles-multistream.xml'
index_name = 'enwiki-20190801-pages-articles-multistream-index.txt'
xml_path = base_path + xml_name
xml_bz2_path = xml_path + '.bz2'
index_path = base_path + index_name
index_bz2_path = index_path + '.bz2'

#### Get page indices

In [ ]:
indices = {}
with open(index_path, 'r', encoding = 'utf-8') as f:
    for line in f:
        [n1, n2, name] = line.strip().split(':',2)
        indices[name] = {'offset': n1, 'id': n2} #indices.append((n1, n2, name))

In [40]:
x = list(indices.keys())
x[:5]

['AccessibleComputing',
 'Anarchism',
 'AfghanistanHistory',
 'AfghanistanGeography',
 'AfghanistanPeople']

#### Find pages

In [102]:
page_id = 12
byte_offset = 615

In [103]:
data = b''
page_save = False
page_id_str = b'<id>' + str(page_id).encode('utf-8') + b'</id>'
with bz2.BZ2File(xml_bz2_path, 'rb') as file:
    file.seek(byte_offset)
    for _ in range(10000):
        line = file.readline()
        if b'<page>' in line:
            data = b''
        data = data + line
        if page_id_str in line:
            page_save = True
        if b'</page>' in line:
            if page_save:
                break
data[:100]

b'  <page>\n    <title>Anarchism</title>\n    <ns>0</ns>\n    <id>12</id>\n    <revision>\n      <id>908640'

#### Parse XML

In [115]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            if self._current_tag == 'id' and self._buffer:
                return
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'id'):
            if name == 'id' and name in self._values.keys():
                return
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)
        if name == 'page':
            self._pages.append((self._values['title'], self._values['text'], self._values['id']))

In [116]:
handler = WikiXmlHandler()
parser = xml.sax.make_parser()
parser.setContentHandler(handler)
for line in data.split(b'\n'):
    parser.feed(line)
handler._pages[0][0]

'Anarchism'

#### Parse wiki

In [118]:
wiki = mph.parse(handler._pages[0][1])

In [119]:
wikilinks = [x.title for x in wiki.filter_wikilinks()]
print(f'There are {len(wikilinks)} wikilinks.')
wikilinks[:5]

There are 519 wikilinks.


['Anti-authoritarianism',
 'political philosophy',
 'Hierarchy',
 "Workers' self-management",
 'Self-governance']